In [1]:
!wget https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip
!wget https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip


--2022-11-13 18:42:44--  https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip
Resolving btsd.ethz.ch (btsd.ethz.ch)... 129.132.52.168, 2001:67c:10ec:36c2::168
Connecting to btsd.ethz.ch (btsd.ethz.ch)|129.132.52.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174298785 (166M) [application/zip]
Saving to: ‘BelgiumTSC_Training.zip’

BelgiumTSC_Training 100%[===================>] 166.22M  6.38MB/s    in 27s     

2022-11-13 18:43:12 (6.16 MB/s) - ‘BelgiumTSC_Training.zip’ saved [174298785/174298785]

--2022-11-13 18:43:12--  https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip
Resolving btsd.ethz.ch (btsd.ethz.ch)... 129.132.52.168, 2001:67c:10ec:36c2::168
Connecting to btsd.ethz.ch (btsd.ethz.ch)|129.132.52.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77877833 (74M) [application/zip]
Saving to: ‘BelgiumTSC_Testing.zip’

BelgiumTSC_Testing. 100%[===================>]  74.27M  6.24MB/s    in 13s     


In [3]:
import zipfile
with zipfile.ZipFile('/content/BelgiumTSC_Training.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [5]:
import torch
import numpy as np
import torchvision.transforms as transforms
import torchvision
import torch.utils.data as td
import matplotlib.pyplot as plt
import torch.nn as nn
from PIL import Image
import torch.nn.functional as F
import gc
import torchvision.datasets as datasets



preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    ),
])
train_path = "/content/Training"
#test_path = "/content/Testing"

#data set from https://btsd.ethz.ch/shareddata/
data = torchvision.datasets.ImageFolder(root=train_path, transform=preprocess)

#train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True,  num_workers=0)

train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True,  num_workers=0)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True,  num_workers=0)

In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 62):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

    num_classes = 62
num_epochs = 20
batch_size = 16
learning_rate = 0.01
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_data_loader)
training_accuracies = []
training_losses = []
test_accuracies = []
test_losses = []

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_data_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #accuracy
    total = labels.size(0)
    _,predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()
    accuracy = (correct / total) * 100

    del images, labels, outputs
    torch.cuda.empty_cache()
    gc.collect()
    accuracy = (correct / total) * 100
    training_accuracies.append(accuracy)
    training_losses.append(loss.item())

    print ('Training set: Epoch [{}/{}], Loss: {:.4f},  Accuracy: {:.2f}%, ' 
                .format(epoch+1, num_epochs, loss.item(), accuracy))

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss_test = criterion(outputs, labels)
            del images, labels, outputs
        accuracy = (correct / total) * 100    
        test_accuracies.append(accuracy)
        test_losses.append(loss_test.item())
        print ('Test set: Epoch [{}/{}], Loss: {:.4f},  Accuracy: {:.2f}%, ' 
                    .format(epoch+1, num_epochs, loss_test.item(), accuracy))

Training set: Epoch [1/20], Loss: 2.5078,  Accuracy: 41.67%, 
Test set: Epoch [1/20], Loss: 2.7122,  Accuracy: 33.22%, 
Training set: Epoch [2/20], Loss: 1.6949,  Accuracy: 50.00%, 
Test set: Epoch [2/20], Loss: 1.7247,  Accuracy: 55.63%, 
Training set: Epoch [3/20], Loss: 1.0967,  Accuracy: 75.00%, 
Test set: Epoch [3/20], Loss: 1.2546,  Accuracy: 60.44%, 
Training set: Epoch [4/20], Loss: 0.6641,  Accuracy: 91.67%, 
Test set: Epoch [4/20], Loss: 1.7590,  Accuracy: 77.49%, 
Training set: Epoch [5/20], Loss: 0.5184,  Accuracy: 83.33%, 
Test set: Epoch [5/20], Loss: 0.7061,  Accuracy: 82.84%, 
Training set: Epoch [6/20], Loss: 1.0672,  Accuracy: 66.67%, 
Test set: Epoch [6/20], Loss: 2.0314,  Accuracy: 68.31%, 
Training set: Epoch [7/20], Loss: 0.7900,  Accuracy: 75.00%, 
Test set: Epoch [7/20], Loss: 0.4725,  Accuracy: 81.09%, 
Training set: Epoch [8/20], Loss: 0.5016,  Accuracy: 75.00%, 
Test set: Epoch [8/20], Loss: 0.6602,  Accuracy: 86.23%, 
Training set: Epoch [9/20], Loss: 0.3504